<a href="https://colab.research.google.com/github/wmendozaFS/Pandas/blob/main/10EJ02_07_Pandas_MySQL_y_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><u><H1>MySQL y Pandas</H1></u></center>

## Installation:
- pip install PyMySQL

In [ ]:
!pip install PyMySQL

Defaulting to user installation because normal site-packages is not writeable


## Abrir conexión

In [ ]:
import pandas as pd
import pymysql

# Parámetros de conexión
host = 'localhost'
port = 3306
user = 'root'
password = '180690Sql_'


# Conexión a MySQL sin especificar base de datos
connection = pymysql.connect(host=host,
                             port=port,
                             user=user,
                             password=password)

##  Crear base de datos

In [ ]:
# Crear cursor
cursor = connection.cursor()

# Crear base de datos si no existe
cursor.execute("CREATE DATABASE IF NOT EXISTS pandas")

# Cerrar conexión
cursor.close()
connection.close()





## Crear tabla datos

In [ ]:
# Conexión a MySQL
database= 'pandas'
connection = pymysql.connect(host=host,
                             port=port,
                             user=user,
                             password=password,
                             database=database)

# Crear cursor
cursor = connection.cursor()

# Crear tabla de usuarios si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS datos_climaticos (
    id INT AUTO_INCREMENT PRIMARY KEY,
    fecha DATE NOT NULL,
    num_estacion INT NOT NULL,
    precipitacion FLOAT,
    temperatura FLOAT,
    humedad FLOAT
);
"""

cursor.execute(create_table_query)

# Cerrar conexión
cursor.close()
connection.close()


In [ ]:
import pymysql
import random
from datetime import datetime, timedelta



# Conexión a MySQL
connection = pymysql.connect(host=host,
                             port=port,
                             user=user,
                             password=password,
                             database=database)

# Crear cursor
cursor = connection.cursor()

def generate_random_data():
    fecha = datetime.now() - timedelta(days=random.randint(0, 365))
    num_estacion = random.randint(1, 10)  # Suponiendo que hay 10 estaciones
    precipitacion = round(random.uniform(0, 150), 2)  # Precipitación entre 0 y 150 mm
    temperatura = round(random.uniform(-10, 40), 2)  # Temperatura entre -10 y 40 grados Celsius
    humedad = round(random.uniform(0, 100), 2)  # Humedad entre 0% y 100%

    return fecha, num_estacion, precipitacion, temperatura, humedad

def insert_random_data():
    for _ in range(10000):
        data = generate_random_data()
        insert_query = """
        INSERT INTO datos_climaticos (fecha, num_estacion, precipitacion, temperatura, humedad)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, data)
    connection.commit()

insert_random_data()

# Cerrar conexión
cursor.close()
connection.close()


1. datetime.now():

Esta función devuelve la fecha y hora actuales. Por ejemplo, si hoy es el 12 de octubre de 2023 a las 3:00 p.m., datetime.now() devolverá 2023-10-12 15:00:00.
timedelta(days=random.randint(0, 365)):

2. timedelta
es una clase de la biblioteca datetime que representa una duración, es decir, la diferencia entre dos fechas o tiempos.
random.randint(0, 365) genera un número entero aleatorio entre 0 y 365 (inclusive). Por lo tanto, timedelta(days=random.randint(0, 365)) genera una duración aleatoria entre 0 y 365 días.

3. datetime.now() - timedelta(...):

Esta operación resta la duración aleatoria generada en el paso anterior de la fecha y hora actuales. El resultado es una fecha que está entre el día actual y hasta 365 días en el pasado.
Por ejemplo, si hoy es el 12 de octubre de 2023 y random.randint(0, 365) genera el número 100, entonces la fecha resultante será el 4 de julio de 2023.

## Leer tabla y crear df

In [ ]:
import pandas as pd
import pymysql


# Establecer conexión a MySQL
connection = pymysql.connect(host=host,
                             port=port,
                             user=user,
                             password=password,
                             database=database)

# Leer la tabla y pasarlo a DataFrame
query = "SELECT * FROM datos_climaticos"
df = pd.read_sql(query, connection)

# Cerrar conexión
connection.close()

# Mostrar las primeras filas del DataFrame
print(df.head())


   id       fecha  num_estacion  precipitacion  temperatura  humedad
0   1  2022-11-15            10         140.36        15.30    24.23
1   2  2023-01-25             2         124.15        -8.15    84.19
2   3  2023-03-23             5         100.66        34.97    75.36
3   4  2022-11-21             6          54.87        -8.85    38.88
4   5  2023-03-07             6          73.21        17.70    83.96


C:\Users\PC\AppData\Local\Temp\ipykernel_35772\4188291963.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [ ]:
df2 = df.copy()
df2

,id,fecha,num_estacion,precipitacion,temperatura,humedad
0,1,2022-11-15,10,140.36,15.30,24.23
1,2,2023-01-25,2,124.15,-8.15,84.19
2,3,2023-03-23,5,100.66,34.97,75.36
3,4,2022-11-21,6,54.87,-8.85,38.88
4,5,2023-03-07,6,73.21,17.70,83.96
...,...,...,...,...,...,...
9995,9996,2023-07-24,7,124.00,2.91,14.13
9996,9997,2023-07-24,10,71.16,35.90,41.91
9997,9998,2023-09-27,8,109.31,9.58,39.29
9998,9999,2023-02-15,4,23.58,38.12,33.82


## Indice sensación térmica

índice de calor (o sensación térmica) basado en la temperatura y la humedad. El índice de calor es una medida de cómo se siente realmente la temperatura cuando se considera la humedad.
La fórmula simplificada del índice de calor es:

Indice de Calor=T+0.33×P−0.70×V−4.00

Donde:

T es la temperatura en grados Celsius.

P es la humedad relativa en porcentaje.

V es la velocidad del viento en m/s (para este ejemplo, asumiremos una velocidad constante de 1 m/s).

In [ ]:
def indice_calor(temperatura, humedad):
    # Asumimos una velocidad del viento constante de 1 m/s
    velocidad_viento = 1.0

    # Calculamos el índice de calor usando la fórmula
    indice = temperatura + 0.33 * humedad - 0.70 * velocidad_viento - 4.00

    return indice

# Ejemplo de uso
temperatura = 30  # 30°C
humedad = 70      # 70% de humedad relativa

sensacion_termica = indice_calor(temperatura, humedad)
print(f"Con una temperatura de {temperatura}°C y una humedad del {humedad}%, la sensación térmica es de {sensacion_termica:.2f}°C.")


Con una temperatura de 30°C y una humedad del 70%, la sensación térmica es de 48.40°C.


### Añadir columna a df con Indice de sensación termica

In [ ]:
import pandas as pd

# Función para calcular el índice de calor
def indice_calor(temperatura, humedad):
    # Asumimos una velocidad del viento constante de 1 m/s
    velocidad_viento = 1.0
    # Calculamos el índice de calor usando la fórmula
    indice = temperatura + 0.33 * humedad - 0.70 * velocidad_viento - 4.00
    return indice

# Añadir la columna 'sensacion_termica' al DataFrame
df['sensacion_termica'] = df.apply(lambda row: indice_calor(row['temperatura'], row['humedad']), axis=1)

# Mostrar el DataFrame actualizado
df


,id,fecha,num_estacion,precipitacion,temperatura,humedad,sensacion_termica
0,1,2022-11-15,10,140.36,15.30,24.23,18.5959
1,2,2023-01-25,2,124.15,-8.15,84.19,14.9327
2,3,2023-03-23,5,100.66,34.97,75.36,55.1388
3,4,2022-11-21,6,54.87,-8.85,38.88,-0.7196
4,5,2023-03-07,6,73.21,17.70,83.96,40.7068
...,...,...,...,...,...,...,...
9995,9996,2023-07-24,7,124.00,2.91,14.13,2.8729
9996,9997,2023-07-24,10,71.16,35.90,41.91,45.0303
9997,9998,2023-09-27,8,109.31,9.58,39.29,17.8457
9998,9999,2023-02-15,4,23.58,38.12,33.82,44.5806


### Punto de rocio

Punto de Rocío. El punto de rocío es la temperatura a la cual el aire se satura y el vapor de agua comienza a condensarse en gotas (rocío). Es una indicación de la cantidad de humedad en el aire.

Aunque existen varias fórmulas para calcular el punto de rocío basadas en la temperatura y la humedad relativa, una aproximación común es la siguiente:

Td = T - ((100 − RH)/5)


Donde:

Td  es el punto de rocío en grados Celsius.
T es la temperatura en grados Celsius.
RH es la humedad relativa en porcentaje.

Vamos a añadir una columna al DataFrame df con el cálculo del punto de rocío:

In [ ]:
import pandas as pd


# Función para calcular el punto de rocío
def punto_rocio(temperatura, humedad):
    return temperatura - (100 - humedad) / 5

# Añadir la columna 'punto_rocio' al DataFrame
df['punto_rocio'] = df.apply(lambda row: punto_rocio(row['temperatura'], row['humedad']), axis=1)

# Mostrar el DataFrame actualizado
df


,id,fecha,num_estacion,precipitacion,temperatura,humedad,sensacion_termica,punto_rocio
0,1,2022-11-15,10,140.36,15.30,24.23,18.5959,0.146
1,2,2023-01-25,2,124.15,-8.15,84.19,14.9327,-11.312
2,3,2023-03-23,5,100.66,34.97,75.36,55.1388,30.042
3,4,2022-11-21,6,54.87,-8.85,38.88,-0.7196,-21.074
4,5,2023-03-07,6,73.21,17.70,83.96,40.7068,14.492
...,...,...,...,...,...,...,...,...
9995,9996,2023-07-24,7,124.00,2.91,14.13,2.8729,-14.264
9996,9997,2023-07-24,10,71.16,35.90,41.91,45.0303,24.282
9997,9998,2023-09-27,8,109.31,9.58,39.29,17.8457,-2.562
9998,9999,2023-02-15,4,23.58,38.12,33.82,44.5806,24.884


### Índice de Aridez

Índice de Aridez. Este índice se utiliza para determinar la sequedad de un área basada en la relación entre la precipitación y la temperatura. Aunque hay varias formas de calcular el índice de aridez, una fórmula simplificada es:

ndice de Aridez= P / (T+10)

Donde:

- P es la precipitación anual en mm.
- T es la temperatura media anual en grados Celsius.

El índice resultante puede ayudar a categorizar el clima de una región. Por ejemplo:

- IA > 0.65: Clima húmedo
- 0.5 < IA ≤ 0.65: Clima subhúmedo
- 0.2 < IA ≤ 0.5: Clima semiárido
- IA ≤ 0.2: Clima árido

Vamos a añadir una columna al DataFrame df con el cálculo del índice de aridez:


In [ ]:
import pandas as pd
import numpy as np  # Importamos numpy para usar NaN

# Función corregida para calcular el índice de aridez
def indice_aridez(precipitacion, temperatura):
    divisor = temperatura + 10
    if divisor == 0:
        return np.NaN  # Devolvemos NaN si el divisor es cero
    return precipitacion / divisor

# Añadir la columna 'indice_aridez' al DataFrame
df['indice_aridez'] = df.apply(lambda row: indice_aridez(row['precipitacion'], row['temperatura']), axis=1)

# Mostrar el DataFrame actualizado
df



,id,fecha,num_estacion,precipitacion,temperatura,humedad,sensacion_termica,punto_rocio,indice_aridez
0,1,2022-11-15,10,140.36,15.30,24.23,18.5959,0.146,5.547826
1,2,2023-01-25,2,124.15,-8.15,84.19,14.9327,-11.312,67.108108
2,3,2023-03-23,5,100.66,34.97,75.36,55.1388,30.042,2.238381
3,4,2022-11-21,6,54.87,-8.85,38.88,-0.7196,-21.074,47.713043
4,5,2023-03-07,6,73.21,17.70,83.96,40.7068,14.492,2.642960
...,...,...,...,...,...,...,...,...,...
9995,9996,2023-07-24,7,124.00,2.91,14.13,2.8729,-14.264,9.604957
9996,9997,2023-07-24,10,71.16,35.90,41.91,45.0303,24.282,1.550327
9997,9998,2023-09-27,8,109.31,9.58,39.29,17.8457,-2.562,5.582737
9998,9999,2023-02-15,4,23.58,38.12,33.82,44.5806,24.884,0.490025


In [ ]:
def clasificar_clima(ia):
    if ia > 0.65:
        return "Clima húmedo"
    elif 0.5 < ia <= 0.65:
        return "Clima subhúmedo"
    elif 0.2 < ia <= 0.5:
        return "Clima semiárido"
    else:
        return "Clima árido"

# Añadir la columna 'clima' al DataFrame basada en la columna 'indice_aridez'
df['clima'] = df['indice_aridez'].apply(clasificar_clima)

# Mostrar el DataFrame actualizado
df


,id,fecha,num_estacion,precipitacion,temperatura,humedad,sensacion_termica,punto_rocio,indice_aridez,clima
0,1,2022-11-15,10,140.36,15.30,24.23,18.5959,0.146,5.547826,Clima húmedo
1,2,2023-01-25,2,124.15,-8.15,84.19,14.9327,-11.312,67.108108,Clima húmedo
2,3,2023-03-23,5,100.66,34.97,75.36,55.1388,30.042,2.238381,Clima húmedo
3,4,2022-11-21,6,54.87,-8.85,38.88,-0.7196,-21.074,47.713043,Clima húmedo
4,5,2023-03-07,6,73.21,17.70,83.96,40.7068,14.492,2.642960,Clima húmedo
...,...,...,...,...,...,...,...,...,...,...
9995,9996,2023-07-24,7,124.00,2.91,14.13,2.8729,-14.264,9.604957,Clima húmedo
9996,9997,2023-07-24,10,71.16,35.90,41.91,45.0303,24.282,1.550327,Clima húmedo
9997,9998,2023-09-27,8,109.31,9.58,39.29,17.8457,-2.562,5.582737,Clima húmedo
9998,9999,2023-02-15,4,23.58,38.12,33.82,44.5806,24.884,0.490025,Clima semiárido


## crear tabla con df

In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Supongamos que tienes el DataFrame df ya definido

# Parámetros de conexión
host = 'localhost'
port = 3306
user = 'root'
password = '180690Sql_'
database = 'pandas'

# Crear conexión con SQLAlchemy
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# Crear una nueva tabla llamada 'clima_data' a partir del DataFrame
df.to_sql('clima_data', con=engine, if_exists='replace', index=False)

print("Tabla 'clima_data' creada con éxito.")


Tabla 'clima_data' creada con éxito.


## leer tabla y crear df

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Parámetros de conexión
host = 'localhost'
port = 3306
user = 'root'
password = '180690Sql_'
database = 'pandas'

# Crear conexión con SQLAlchemy
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# Leer la tabla 'clima_data' y guardarla en df2
df2 = pd.read_sql('clima_data', con=engine)

# Mostrar las primeras filas del DataFrame df2
df2.head()


,id,fecha,num_estacion,precipitacion,temperatura,humedad,sensacion_termica,punto_rocio,indice_aridez,clima
0,1,2022-11-15,10,140.36,15.30,24.23,18.5959,0.146,5.547826,Clima húmedo
1,2,2023-01-25,2,124.15,-8.15,84.19,14.9327,-11.312,67.108108,Clima húmedo
2,3,2023-03-23,5,100.66,34.97,75.36,55.1388,30.042,2.238381,Clima húmedo
3,4,2022-11-21,6,54.87,-8.85,38.88,-0.7196,-21.074,47.713043,Clima húmedo
4,5,2023-03-07,6,73.21,17.70,83.96,40.7068,14.492,2.642960,Clima húmedo


# Pandas y SQLite

1. Instalación y configuración:
Asegúrate de tener pandas instalado:

2. CRUD con pandas y SQLite:
- Create (Crear)

Para crear una nueva tabla y guardar un DataFrame en SQLite:

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Crear un DataFrame de ejemplo
data = {
    'nombre': ['Ana', 'Luis', 'Marta'],
    'edad': [25, 30, 22],
    'ciudad': ['Madrid', 'Barcelona', 'Valencia']
}
df = pd.DataFrame(data)

# Conexión a SQLite
engine = create_engine('sqlite:///mydb_sqlite.db')

# Guardar el DataFrame en una nueva tabla llamada 'personas'
df.to_sql('personas', con=engine, if_exists='replace', index=False)


3

- Read (Leer)

Para leer una tabla de SQLite y guardarla en un DataFrame:

In [ ]:
df2 = pd.read_sql('personas', con=engine)
df2


,nombre,edad,ciudad
0,Ana,25,Madrid
1,Luis,30,Barcelona
2,Marta,22,Valencia


- Update (Actualizar)

Para actualizar registros en SQLite:

In [ ]:
# Supongamos que queremos actualizar la edad de Ana a 26
df2.loc[df2['nombre'] == 'Ana', 'edad'] = 26

# Guardar los cambios en la tabla 'personas'
df2.to_sql('personas', con=engine, if_exists='replace', index=False)


3

- Delete (Eliminar)

Para eliminar registros en SQLite:

In [ ]:
# Supongamos que queremos eliminar a Luis del DataFrame
df2 = df2[df2['nombre'] != 'Luis']

# Guardar los cambios en la tabla 'personas'
df2.to_sql('personas', con=engine, if_exists='replace', index=False)


2